In [8]:
import numpy as np
from pandas import DataFrame
import pandas as pd
from matplotlib import pyplot
from sklearn.preprocessing import KBinsDiscretizer
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.tree import _tree
from tkinter import *
from tkinter import messagebox
import tkinter.scrolledtext as st
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import accuracy_score
import warnings; warnings.simplefilter('ignore')

# isDataLoaded=0
# isDiscretize=0
#from tkinter.ttk import *

#df=pd.read_csv("Dry_Bean.csv")
top = Tk()
top.geometry("650x300")
top.title("ADM Project") 
top.configure(bg='white')
#l1 = Label(top,text = "Dry Beans Data Set",bg="black",fg="white").place(x = 140,y = 10)
# the label for Discretize
def LoadData():
    #isDataLoaded=1;
    global df
    df=pd.read_csv("Dry_Bean.csv")
    df.head()
    messagebox.showinfo("Datset is loaded", str(df.columns))
    

def Discretize():
    #if isDataLoaded :
        isDiscretize=1
        X=df.iloc[:, :-1]
        Y=df['Class']
        kbins = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
        data_trans = kbins.fit_transform(X)
        global df1
        df1 = DataFrame(data_trans)
        df1.columns=('Area','Perimeter','MajorAxisLength','MinorAxisLength','AspectRation','Eccentricity','ConvexArea','EquivDiameter','Extent','Solidity','roundness','Compactness','ShapeFactor1','ShapeFactor2','ShapeFactor3','ShapeFactor4')
        df1.head()
        messagebox.showinfo("showinfo", "Data is Discretized")
    #else :
        #messagebox.showinfo("showinfo", "Please Load DataSet First")
    
    
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
            p1 += [f"({name} <= {np.round(threshold, 3)})"]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [f"({name} > {np.round(threshold, 3)})"]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        if(path[-1][1]>100):
            rule = "if "
            for p in path[:-1]:
                if rule != "if ":
                    rule += " and "
                rule += str(p)
            rule += " then "
            if class_names is None:
                rule += "response: "+str(np.round(path[-1][0][0][0],3))
            else:
                classes = path[-1][0][0]
                l = np.argmax(classes)
                rule += f"class: {class_names[l]}(proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
            rule += f" | based on {path[-1][1]:,} samples"
            rules += [rule]

    return rules
    
def Generate_Rules():
    
        
    feature_cols = ['Area','Perimeter','MajorAxisLength','MinorAxisLength','AspectRation','Eccentricity','ConvexArea','EquivDiameter','Extent','Solidity','roundness','Compactness','ShapeFactor1','ShapeFactor2','ShapeFactor3','ShapeFactor4']
    X=df1
    Y=df['Class']
    RDF_Classifier = DecisionTreeClassifier(random_state=0)
    RDF_Classifier.fit(X,Y);
    rules = get_rules(RDF_Classifier,feature_cols,Y.unique())
    ## GUI CODE
    wndRules = Toplevel(top)
    wndRules.title("Rules Window")
    wndRules.geometry("1300x700")
    text_area = st.ScrolledText(wndRules,width = 155,height = 50)
    text_area.grid(column = 0, pady = 10, padx = 10)
    c=0
    for r in rules:
        c=c+1
        #print(r)
        text_area.insert(END,r)
        text_area.insert(END,'\n\n')
    text_area.insert(END,c)
    #print(c)
    
## GA PART ###    
def initialize_population():
    population = np.random.randint(2, size=(10,16))
    features = ['Area','Perimeter','MajorAxisLength','MinorAxisLength','AspectRation','Eccentricity','ConvexArea','EquivDiameter','Extent','Solidity','roundness','Compactness','ShapeFactor1','ShapeFactor2','ShapeFactor3','ShapeFactor4']
    return population,features

def cost(features,chromosome):
    feature_selected = []
    for i in range(len(features)):
        if chromosome[i] == 1:
            feature_selected.append(features[i])
    x = df[feature_selected]
    x_abs=x.abs()
    xMax = x_abs.max()
    x = x/xMax
    y = df[['Class']]
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3,random_state =2)
    clf = DecisionTreeClassifier(criterion="entropy",random_state=2,max_depth=20)
    clf.fit(x_train, y_train)
    t=clf.predict(x_test)
    return accuracy_score(y_test, t)

def random_combine(parents,n_offspring):
    offspring = []
    n_parents = parents.shape[0]
    for i in range(n_offspring):
        random_dad = parents[np.random.randint(low =0,high = n_parents - 1)]
        random_mom = parents[np.random.randint(low =0,high = n_parents - 1)]
        dad_mask = np.random.randint(0,2,np.array(random_dad).shape)
        mom_mask = np.logical_not(dad_mask)
        child = np.add(np.multiply(random_dad,dad_mask),np.multiply(random_mom,mom_mask))
        offspring.append(child)
    offspring = np.array(offspring)
    return offspring

def mutate_parent(parent,n_mutations):
    size1 = parent.shape[0]
    for i in range(n_mutations):
        rand1 = np.random.randint(0,size1)
        if parent[rand1] == 0:
            parent[rand1] = 1
        else:
            parent[rand1] = 0
    return parent

def mutate_gen(parent_gen,n_mutations):
    mutated_parent_gen = []
    for parent in parent_gen:
        mutated_parent_gen.append(mutate_parent(parent,n_mutations))
    mutated_parent_gen = np.array(mutated_parent_gen)
    #print("Mutated population:",mutated_parent_gen)
    return mutated_parent_gen

def select_best(features,parent_gen):
    costs = []
    selected_parent = []
    for i in range(len(parent_gen)):
        costs.append(cost(features,parent_gen[i]))
        #print("Cost",[i],"=",costs[i])
        if costs[i] > 0.4:
            selected_parent.append(parent_gen[i])
            #print("Selected",[i],"=",selected_parent[i])
    #print(costs) 
    costs = np.array(costs)
    selected_parent = np.array(selected_parent)
    #print(selected_parent)
    #print(costs)
    #print(selected_parent.shape)
    max_index = np.where(costs == np.amax(costs))
    feasible_features = (parent_gen[max_index])[0]
    #print("Feasible feature:",feasible_features)
    return selected_parent,costs,feasible_features

def gen_algo():
    ## GUI CODE
    wndRules = Toplevel(top)
    wndRules.title("GA Rules")
    wndRules.geometry("1300x700")
    text_area = st.ScrolledText(wndRules,width = 155,height = 50)
    text_area.grid(column = 0, pady = 10, padx = 10)
    ## GUI END
    X=df.iloc[:, :-1]
    Y=df['Class']
    parent_gen,features = initialize_population()
    print(parent_gen.shape)
    generations = 15
    overall_costs = []
    overall_features = []
    for i in range(generations):
        #print("Generation number :",i+1)
        text_area.insert(END,"Generation number:  "+str(i+1))
        text_area.insert(END,'\n')
        parent_gen,costs,feasible_features = select_best(features,parent_gen)
        #print("Accuracy for this generation:",np.amax(costs))
        text_area.insert(END,"Accuracy for this generation:  "+str(np.amax(costs)))
        text_area.insert(END,'\n')
        #print("Feasible features for this generation:",feasible_features)
        text_area.insert(END,"Feasible features for this generation:  "+str(feasible_features))
        text_area.insert(END,'\n')
       
        overall_costs.append(np.amax(costs))
        overall_features.append(feasible_features)
        if parent_gen.shape[0] <= 1:
            parent_gen,features = initialize_population()
        else :
            parent_gen = random_combine(parent_gen,20)
        parent_gen = mutate_gen(parent_gen,2)
        #print("=============================================================================================================")
        text_area.insert(END,"=============================================================================================================")
        text_area.insert(END,'\n')
    overall_costs = np.array(overall_costs)
    overall_features = np.array(overall_features)
    #best_child,costs,feasible_features = select_best(features,parent_gen)
    max_index = np.where(overall_costs == np.amax(overall_costs))
    #print("Final cost:",np.amax(overall_costs))
    text_area.insert(END,"Final cost:  " +str(np.amax(overall_costs)))
    text_area.insert(END,'\n')
                     
    #print("Final features:",(overall_features[max_index])[0])
    text_area.insert(END,"Final features:  "+str((overall_features[max_index])[0]))
    text_area.insert(END,'\n')
    #return (overall_features[max_index])[0]
    text_area.insert(END,str((overall_features[max_index])[0]))
    

# def FeatureSelUsing_GA():
#     final_feature =gen_algo
    

Label(top,text ="Dry Bean Data Set",height=2,width=20,bg='white',font=("Arial", 25)).pack(pady=1)    
Button(top,text = "LoadDataset", bg="green",fg="white",width=20,height=2,command=LoadData).place(x = 600,y = 100)
Button(top,text = "Feature selection using GA", bg="Blue",fg="white",width=20,height=2,command=gen_algo).place(x = 600,y = 150)
Button(top,text = "Discretize", bg="orange",fg="white",width=20,height=2,command=Discretize).place(x = 600,y = 200)
Button(top,text = "Generate Rules", bg="Red",fg="white",width=20,height=2,command=Generate_Rules).place(x = 600,y = 250)

#l1 = Label(top,text = "Discretize",bg="orange",fg="white").place(x = 40,y = 60)
#text1 = Entry(top,width = 30).place(x = 110,y = 60)
#a = int(text1.get())
#place(x = 90,y = 130)
#def fun():
    
#submit_button = Button(top,text = "Generate Rules", bg="Blue",fg="white").place(x = 90,y = 130)


top.mainloop()


(10, 16)
